# 第六讲：事件模型、神经网络、支持向量机

接着上一讲，进一步讨论朴素贝叶斯算法及事件模型。

## 2.2 文本分类的事件模型

在结束生成学习算法前，我们再来介绍一种针对文本分类的模型。虽然前面介绍的朴素贝叶斯算法对于很多分类问题都要良好的效果，不过对于文本分类，我们有一种更好的模型。

在特定语境的文本分类中，朴素贝叶斯算法使用了**多元伯努利事件模型（multi-variate Bernoulli event model）**（特征值按照伯努利试验取$x_i\in\{0,1\}$，特征值向量长度为字典长度）。回忆一下上一讲的知识，在这个模型中，我们首先假设下一封邮件的发送方已经确定，一个随机的邮件发送人（可能是垃圾邮件制造者，也可能只是普通联系人，是由我们确定的条件概率中的条件）；接着我们假设，发送者会遍历词典，独立的决定是否将单词$x_i$写入邮件（该词语相对于其他词语独立），而且该词语服从概率$p(x_i=1\mid y)=\phi_{i\mid y}$（如果是垃圾邮件制造者则可能更倾向于发送带有“buy”、“sales”、“discount”等单词的 邮件，而普通联系人则会选择更正常的词语分布发送正常邮件）。于是我们得到，$p(x\mid y)=\prod_{i=1}^np(x_i\mid y)$和$p(y)$，再根据$\mathrm{arg}\displaystyle\operatorname*{max}_yp(x\mid y)=\mathrm{arg}\displaystyle\operatorname*{max}_yp(x\mid y)p(y)$最终得到该邮件是垃圾邮件的概率：$p(y)\prod_{i=1}^np(x_i\mid y)$。这个模型存在一些小问题，比如，它不能讲词语的出现的次数反应在输入特征值向量中，如果一封邮件多次出现“buy”这个词，那么此邮件确实更有可能是垃圾邮件。

下面介绍的模型是朴素贝叶斯算法算法的简单变形，称为**多项事件模型（multinomial event model）**。接下来，我们将使用不同的记法和不同的特征值描述邮件。使用$x_i$表示邮件中的第$i$个词语，$x_i$在是一个在$\{1,\cdots,\lvert V\rvert\}$取值的整数，而$\lvert V\rvert$就是词汇表（词典）的大小。于是，一封邮件被我们表示为$(x_1,x_2,\cdots,x_n)$向量，注意，对于不同的邮件$n$是不同的。举个例子，对于一封以“A NIPS”开头的邮件，有$x_1=1$（“a”是词典的第一个单词）、$x_2=35000$（假设“nips”是词典的第35000个单词）。

在多项事件模型中，我们像以前一样假设邮件发送方已经由一个随机过程确定（根据$p(y)$），可能是垃圾邮件制造者，也可能是普通联系人；接着，发件人从某多项分布（$p(x_i\mid y)$）中选出一个词$x_1$；然后根据相同的多项分布选出第二个词$x_2$（独立于$x_1$），继续重复这个动作选出$x_3,x_4,\cdot,x_n$；最后，邮件编写结束。因此，该邮件是垃圾邮件的概率为$p(y)\prod_{i=1}^np(x_i\mid y)$，这个式子和前面的完全一样，但是请注意，这里的特征值$x_i$是邮件第$i$个单词，其取值不再是多元伯努利事件模型中的$\{0,1\}$了，它现在是一个多项分布。

多项事件模型的参数为：$\phi_y=p(y)$（同以前一样），$\phi_{k\mid y=1}=p(x_j=k\mid y=1),\ \phi_{k\mid y=0}=p(x_j=k\mid y=0)$（对任意$j$）。注意到这里我们假设了对于任意$j$都有其$p(x_j\mid y)$相等，即在$y$条件下关于某词语的概率分布与该词语出现在邮件里的位置（$j$）无关。

对于训练集$\left\{\left(x^{(i)},y^{(i)}\right);i=1,\cdots,m\right\}$（其中$x^{(i)}=\left(x_1^{(i)},x_2^{(i)},\cdots,x_{n_i}^{(i)}\right)$代表训练集第$i$封样本邮件特征值向量，邮件共有$n_i$个单词），参数的似然函数为：

$$\begin{align}\mathcal{L}\left(\phi_y,\phi_{k\mid y=0},\phi_{k\mid y=1}\right)&=\prod_{i=1}^mp\left(x^{(i)},y^{(i)}\right)\\&=\prod_{i=1}^m\left(\left(\prod_{j=1}^{n_i}p\left(x_j^{(i)}\mid y;\phi_{k\mid y=1},\phi_{k\mid y=0}\right)\right)p\left(y^{(i)};\phi_y\right)\right)\end{align}$$

最大化似然函数将得到各参数的最大似然估计：

$$\begin{align}\phi_{k\mid y=1}&=p(x_j=k\mid y=1)&=\frac{\sum_{i=1}^m\sum_{j=1}^{n_i}1\left\{x_j^{(i)}=k\land y^{(i)}=1\right\}}{\sum_{i=1}^m1\left\{y^{(i)}=1\right\}n_i}\\\phi_{k\mid y=0}&=p(x_j=k\mid y=0)&=\frac{\sum_{i=1}^m\sum_{j=1}^{n_i}1\left\{x_j^{(i)}=k\land y^{(i)}=0\right\}}{\sum_{i=1}^m1\left\{y^{(i)}=0\right\}n_i}\\\phi_y&=p(y=1)&=\frac{\sum_{i=1}^m1\left\{y^{(i)}=1\right\}}{m}\end{align}$$

我们对$\phi_{k\mid y=0},\phi_{k\mid y=1}$应用拉普拉斯平滑（在实际问题中应用拉普拉斯平滑通常可以得到更好的模型），即在分子上加一，分母上加$\lvert V\rvert$，得到：

$$\begin{align}\phi_{k\mid y=1}&=\frac{\left(\sum_{i=1}^m\sum_{j=1}^{n_i}1\left\{x_j^{(i)}=k\land y^{(i)}=1\right\}\right)+1}{\left(\sum_{i=1}^m1\left\{y^{(i)}=1\right\}n_i\right)+\lvert V\rvert}\\\phi_{k\mid y=0}&=\frac{\left(\sum_{i=1}^m\sum_{j=1}^{n_i}1\left\{x_j^{(i)}=k\land y^{(i)}=0\right\}\right)+1}{\left(\sum_{i=1}^m1\left\{y^{(i)}=0\right\}n_i\right)+\lvert V\rvert}\end{align}$$

在处理文本分类问题时，多项事件模型通常比原始的朴素贝叶斯算法效果更好，一个可能的原因是因为它考虑了每个单词出现的次数。

尽管朴素贝叶斯分类器不是最好的分类算法，但它的效果一般都非常好，再加上它简单且易于实现的特性，我们通常用它作为“首选试验算法”。（使用朴素贝叶斯算法最终会得到一个逻辑函数形式的后验分布，也就是说，朴素贝叶斯算法也属于指数分布族，它仍然是一个线性分类器。）